Mean Reward: The primary goal is to maximize the reward achieved by the model during RL training. 

Objective KL Divergence: KL divergence (Kullback-Leibler divergence) measures the dissimilarity between two probability distributions.

In [1]:
import torch
from transformers import GPT2Tokenizer

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer

In [2]:
# load a model. 
model = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
# This is going to be 2nd model, only for reference
model_ref = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
# Common tokenizer 
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [3]:
# bring on the trainer
ppo_config = {"mini_batch_size": 1,
              "batch_size": 1}
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(config, model, model_ref, tokenizer)
# trainer takes the config, two models and tokenizer

/home/kamal/jupyter_env/lib/python3.11/site-packages/trl/trainer/ppo_trainer.py:235: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [ ]:
config  # there is lot more default config than I imagined

In [ ]:
model.pretrained_model.device

In [4]:
query_txt = "This morning I went to the "
query_tensor = tokenizer.encode(query_txt, return_tensors='pt').to(model.pretrained_model.device)

In [ ]:
query_tensor

In [ ]:
# Testing the model functionality
test_model_output = model.generate(query_tensor, max_new_tokens=50)
model_output_text = tokenizer.decode(test_model_output[0])
model_output_text

In [ ]:
tokenizer.eos_token

In [ ]:
query_tensor.shape

In [ ]:
[x for x in query_tensor]

In [7]:
torch.sum(query_tensor[:, -1])  # query_tensor[:, -1] = tensor([220], device='cuda:0')

tensor(220, device='cuda:0')

In [5]:
gen_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token,
    "max_new_tokens": 20
}

In [6]:
response_tensor = ppo_trainer.generate([item for item in query_tensor],
                                       return_prompt=False,
                                       **gen_kwargs)

response_text = tokenizer.decode(response_tensor[0])

TypeError: sum(): argument 'input' (position 1) must be Tensor, not bool

In [ ]:
query_tensor[0]

In [ ]:
# This is the reward and train loop...
reward = [torch.tensor(1.0, device=model.pretrained_model.device)]

# 6. train model with ppo
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

In [ ]:
# push the model on the Hub
model.push_to_hub("fine_tune_gpt_ppo")

# or save it locally
model.save_pretrained("fine_tune_gpt_ppo")

# load the model from the Hub, the model is loaded with AutoModel directly
# not the TRL class
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("my-fine-tuned-model-ppo")

In [ ]:
# 2. Create optimizer
import bitsandbytes as bnb

sgd_optimizer = torch.optim.SGD(model.parameters(), lr=config.learning_rate)

adam_optimizer = bnb.optim.Adam8Bit(model.parameters(), lr=config.learning_rate)


In [ ]:
# PPO_trainers with different optimizers
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(sgd_optimizer, gamma=0.9)
updated_trainer_sgd = PPOTrainer(ppo_config, model,
                                 model_ref, tokenizer,
                                 optimizer=sgd_optimizer,
                                 lr_scheduler=lr_scheduler)

updated_trainer_adam = PPOTrainer(ppo_config, model, model_ref, tokenizer, optimizer=adam_optimizer)